In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import json
import torch
import numpy as np
import time
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation 
from IPython.display import clear_output
from nerfstudio.utils.eval_utils import eval_setup
from inerf.inerf_trainer import load_data_into_trainer
from inerf.inerf_utils import get_corrected_pose, load_eval_image_into_pipeline, get_relative_pose, get_absolute_diff_for_pose, get_image

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
os.chdir('/workspace')
MODEL_PATH = "/workspace/outputs/jackal_with_box/plane-nerf-simplified/2024-02-13_163941"
DATA_PATH = "/stored_data/jackal_with_box_eval/"
GROUND_TRUTH_PATH = os.path.join(DATA_PATH, "ground_truth.json")
TRANSFORM_FILE = "transforms_10.json"
with open(GROUND_TRUTH_PATH) as f:
    GROUND_TRUTH = json.load(f)
with open(os.path.join(DATA_PATH, TRANSFORM_FILE)) as f:
    TRANSFORM = json.load(f)
SAVE_FILE_NAME = "eval_results_"+str(time.strftime("%Y-%m-%d_%H%M%S"))+".csv"

In [4]:
config_path = os.path.join(MODEL_PATH, "config.yml")
config, pipeline, _, _ = eval_setup(
                        Path(config_path),
                        test_mode="inference",
                        )
config.pipeline.datamanager.pixel_sampler.num_rays_per_batch = 4096 

train_loop = 10
n = 100
lr_max = 5e-3
lr_min = 1e-4

Loading latest checkpoint from load_dir

✅ Done loading checkpoint from 
outputs/jackal_with_box/plane-nerf-simplified/2024-02-13_163941/nerfstudio_models/step-000009999.ckpt

In [5]:
output = []

for f in range(len(TRANSFORM["frames"])):
    transform = TRANSFORM.copy()
    transform["frames"] = [TRANSFORM["frames"][f]]
    
    tf = GROUND_TRUTH["frames"][f]["transform_matrix"]
    tf = np.asarray(tf)
    tf = tf[:3, :4 ]
    ground_truth_poses = [tf]
    ground_truth_poses = torch.tensor(ground_truth_poses).to(pipeline.device)
    
    pipeline = load_eval_image_into_pipeline(pipeline,DATA_PATH,transform)

    config.pipeline.datamanager.pixel_sampler.num_rays_per_batch = 4096 

    trainer = load_data_into_trainer(
        config,
        pipeline,
        plane_optimizer = False
    )
    
    clear_output(wait=True)
    print(f)
    
    trainer.pipeline.datamanager.KERNEL_SIZE = 5
    trainer.pipeline.datamanager.THRESHOLD = 5
    trainer.pipeline.datamanager.METHOD = "sift"
    trainer.pipeline.datamanager.get_inerf_batch()  
        
    store = torch.tensor([])

    corrected_pose = get_corrected_pose(trainer)

    relative_pose = get_relative_pose(ground_truth_poses, corrected_pose)
    t_diff, r_diff = get_absolute_diff_for_pose(relative_pose)

    store = torch.cat((store, torch.tensor([torch.mean(t_diff), torch.mean(r_diff)])), 0)

    for i in range(train_loop):
        for j in range(n):
            lr = lr_min + (lr_max - lr_min) * (i / train_loop)
            trainer.pipeline.train()
            loss = trainer.train_iteration_inerf(i*n + j,optimizer_lr = 1e-3)
        corrected_pose = get_corrected_pose(trainer)

        relative_pose = get_relative_pose(ground_truth_poses, corrected_pose)
        t_diff, r_diff = get_absolute_diff_for_pose(relative_pose)

        store = torch.cat((store, torch.tensor([torch.mean(t_diff), torch.mean(r_diff)])), 0)
    
    output.append(store.to("cpu").numpy().tolist())
    np.savetxt(os.path.join(MODEL_PATH,SAVE_FILE_NAME), np.asarray(output), delimiter=",")

45
Number of keypoints:  49
Number of rays:  2326
Randomly select more rays
Final number of rays:  4096
